<a href="https://colab.research.google.com/github/bruzerravi/Hello-World/blob/master/XF_AERO_Ravi9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[73]:<br>
Last successful run to completion on May 21, 2022<br>
This code works only for NACA 4 digit airfoils and output generated by Xflr5    

Ravi9 is developed from Ravi8 to have Cp (99 values)  Re, Alfa, camber, camber_xmax, thickness and turb parameters<br>
(together named as fv) as input and airfoil y-coordinates as output.  

This code is developed from Ravi8 version <br>
The code reads an output file created by XF_AERO_DataRead9 which looks like <br>
DataSets\\Cp_Graph_nacaxxxx_Alfa-m6-10-1_Re-10-100-10_thk-2-12-2_turb_yp_out.csv

The code XF_AERO_DataRead9 takes as its input the (xflr5 created)<br>
"DataSets\\Cp_Graph_nacaxxxx_Alfa-m6-10-1_Re-10-100-10_thk-2-12-2.csv" file and outputs  <br>
"DataSets\\Cp_Graph_nacaxxxx_Alfa-m6-10-1_Re-10-100-10_thk-2-12-2_turb_yp_out.csv"<br>
which is the input file for this code Ravi9

The training set is now in the proper XY form but is read as a dataframe<br>
which must be processed to recreate the XY data for the ANN model

This version of the code checks for consistency of the training set data <br>
read from the input file

In [ ]:
 
# This version is still incomplete: The save model option implemented here does not work
from google.colab import drive
drive.mount('/content/drive')

The XY dataset created from the input dataset after random shuffling is written to a .csv file<br>
<br>
The only input files are the *_out.csv and Profiles_nacaxxxx.csv <br>
These are the only two inputs to run this code All other parameters are detrmined from these inputs<br>
The number of training and test examples may be input as seen below

In [ ]:
import os
import time
import datetime

mport tensorflow as tf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
localtime = time.asctime( time.localtime(time.time()) )
print ("\n \n ***********************************Local start time :", localtime, "***********************************")
print ("\n \n ******************************************** Executing XF_AERO_Ravi9 ********************************************")
print ("\n \n *********************  Input data for this code is generated by executing XF_AERO_DataRead9 *********************")
print ("\n \n **** XF_AERO_Ravi9 predicts profile coordinates for test cases given Cp distribution on Airfoil and the flow parameters***")

---------------------------------------------------------------------------<br>
File names defined

In [ ]:
data_file_in_fv = '/content/drive/MyDrive/DataSets/Cp_Graph_naca44xx_Alfa-m6-10-1_Re-10-100-10_thk-2-12-2_turb_yp_out.csv'
data_file_af_case = data_file_in_fv[18:len(data_file_in_fv)-8]
data_file_out_yp = data_file_af_case +"_out.xlsx"
data_file_in_Pr = '/content/drive/MyDrive/DataSets/Profiles_naca44xx.csv'
!ls /content/drive/MyDrive/DataSets

In [ ]:
AF_family_name = data_file_in_fv[41:49]
n_test = 40
n_train = 50
n_beg = 21          # The training example serial number to start extracting from
n_end = n_beg + n_train
print ("\n \n ** The Input data file for this run is",data_file_in_fv, "**")

----------------------------------------------------------------------------<br>
The calling function is<br>
print_results(predictions, YY, X_trng_par_list, X_trng_af, Cp_trng_set, File_name)

Consolidate the reults matrix

Write the results Dataset to an .xlsx file

In [ ]:
def print_results(predictions, YY, X_par_list, X_af, Cp_set, file_name):
    
    np.set_printoptions(precision=3)
    
    file_name = "Ravi9_" + file_name    
    results = np.zeros((1,YY.shape[1]))
    results[0,:] = Cp_set[0,:]
    results = np.concatenate((results, YY), axis=0)
    results = np.concatenate((results, predictions), axis=0)
    results = np.concatenate((results, Cp_set[1:,:]), axis=0)    
    length_Xaf = len(X_af)
# Collect Alfa, Re, thk results in an array to be written out in a file
    length_Xpar = int(len(X_par_list)/2)         #divide by 2 because the length of X_par_list is double of X_af
    row_labels = []
    row_labels.append('x_p')
    index = np.arange(0,len(X_par_list),1)
    for i in index:
        stringi = X_par_list[i]
        row_labels.append(X_par_list[i])
   
    
    index = np.arange(1,length_Xaf+1,1)
    for i in index:
        string_af = X_af[i-1]
        row_labels[i] = string_af + '-' + row_labels[i]
    index = np.arange(1,length_Xaf+1,1)
    for i in index:
        string_af = X_af[i-1]
        row_labels[length_Xaf+i] = string_af + row_labels[length_Xaf+i]
    len(row_labels)
    index = np.arange(0,length_Xpar,1)
    for i in index:
        stringi = X_par_list[i]
        string1 = stringi[0:61] + 'Cp' + stringi[63:]
        row_labels.append(string1)
    index = np.arange(1,length_Xaf+1,1)
    for i in index:
        string_af = X_af[i-1]
        row_labels[2*length_Xpar+i] = string_af + row_labels[2*length_Xpar+i]
    df_results = pd.DataFrame(data=results, index=row_labels)
    df_results.to_excel(file_name)
    return

----------------------------------------------------------------------------<br>
Calling function and arguments<br>
plot_results(predictions, YY, X_test_par_list, X_test_af, Cp_test_set)
Function definition to plot results

Function definition to plot results

Consolidate the results matrix

Create a dataframe of the results that can be plotted as graphs

In [ ]:
def plot_results(Y_p, Y_a, X_par_list, X_af, Cp_set):
    
    length_Xaf = len(X_af)
    Y_size = Y_p.shape
    results = np.zeros((1,Y_a.shape[1]))
    results[0,:] = Cp_set[0,:]                                  # sets the x-coordinates
    results = np.concatenate((results, Cp_set[1:,:]), axis=0)   # the Cp distribution of the profile    
    results = np.concatenate((results, Y_a), axis=0)            # Actual yp distribution
    results = np.concatenate((results, Y_p), axis=0)            # predicted yp distribution
    df_results = pd.DataFrame(data=results)
    yp_err = np.zeros((1,Y_size[1]))
    i_count = 0                         # count of number of test airfoils
    while (i_count <= Y_size[0]-1):
        row_labels = ["Airfoil Profile", "yp_Actual", "yp_pred", "yp_err"]
        yp_err = np.array(df_results.iloc[Y_size[0] + i_count+1, :]-df_results.iloc[2*Y_size[0] + i_count+1, :])
        Title1 = X_par_list[i_count]
        Title2 = X_af[i_count] + "_" + Title1[0:53] + Title1[64:]
        plt.figure(figsize=(15,7))
        plt.plot(df_results.iloc[0, :], df_results.iloc[i_count+1, :])
        plt.plot(df_results.iloc[0, :], df_results.iloc[Y_size[0] + i_count+1, :], marker='+')
        plt.plot(df_results.iloc[0, :], df_results.iloc[2*Y_size[0] + i_count+1, :], marker='*')
        plt.plot(df_results.iloc[0, :], yp_err[:], marker = 'o')
        tit = Title2 + " : Airfoil Cp Distribution and Profile" 
        plt.title(tit)
        plt.xlabel('x')
        plt.ylabel('y, Cp')
        plt.legend(row_labels, loc=1, prop={'size': 8})
        i_count += 1
    
    return

In [ ]:
def string_list_from_X_par(X_par_list):
# Generate string list from X_par_list
# Merge all entries in array column to a string
    jrange = X_par_list.shape[1]
    irange = X_par_list.shape[0]
    si = []
    i=0
    while (i<= irange-1):
        j=0
        sj = " "
        while (j<= jrange-1):
            if j< jrange-1:
                sj =  sj  + str(X_par_list[i,j]) + " , "
            else:
                sj =  sj  + str(X_par_list[i,j])
            j = j+1
        si.append(sj)
        i = i+1
    
    i=0
    while (i<= irange-1):
        si[i] = "(Alfa , Re, cmbr, cmbrx, thk,  NCrit, XTrTop, XtrBot)-actual-yp =" + si[i]
        i +=1
    
    i = irange
    k=0
    while (i<= 2*irange-1):
        string = si[k]
        sj = "(Alfa , Re, cmbr, cmbrx, thk, NCrit, XTrTop, XtrBot)-pred-yp =" + string[65:]
        si.append(sj)
        i +=1
        k +=1
    return si

Read the input data frame consisting of the training set<br>
Read the input data frame consisting of the training set<br>
Note that each row of the input dataframe has n_Cp_var (=99) values followed by <br>
n_par_var (=8) parameter variables and then n_yp_var (=99=n_yp_var) coordinate values

In [ ]:
df = pd.read_csv(data_file_in_fv)
df = df.sample(frac=1)
df_Pr = pd.read_csv(data_file_in_Pr)
af_trng_set = []
af_trng_set = list(df.iloc[:,0])

delete the first column from the dataframe read since that is only the row serial #

In [ ]:
df = df.iloc[:,1:]
df_size = df.shape
af_family = data_file_af_case[0:8] 
dfPr_size = df_Pr.shape
n_Cp_var=dfPr_size[0]
n_yp_var = n_Cp_var

save the x-y set for each profile.  Note that the x-coordinate is same for all profiles

In [ ]:
n_profiles = int(dfPr_size[1]/2)
x_y_prof_coords = np.zeros((n_yp_var, n_profiles+1))
x_y_prof_coords[:,0:2] = df_Pr.iloc[:,0:2]
i = 3
j = 2
while (i <= 2*n_profiles-1):
    x_y_prof_coords[:,j] = df_Pr.iloc[:,i]
    i +=2
    j +=1
x_y_prof_coords = np.transpose(x_y_prof_coords)
x_coord = np.zeros((1,n_yp_var))
x_coord[0,:] = x_y_prof_coords[0,:]
n_out_var = n_yp_var
n_in_var = df_size[1] - n_out_var
n_par_var = n_in_var - n_Cp_var

denote feature variable by fv_data

In [ ]:
fv_last = n_in_var-1        # Note that this is the column index of last fv value in any row
col_last = df_size[1]-1     # Note that this is the column index of last value in any row
print('\n', "          Feature variables are the upper and lower surface fv data, Alfa , Re, thk,  NCrit, XTrTop, XtrBot")
print("           size of Training Set is ", df_size[0], '\n')
print("           length of dataframe is ", df_size[1], '\n')
print("           size of fv_data is ", df_size[1]-n_out_var, '\n')
print("           the fv value in row #0 col #0 is  df.iloc[0,0] =", df.iloc[0,0], '\n')
print("           the last fv value in row #0 col #last is  df.iloc[0,last] =", df.iloc[0,fv_last], '\n')
print("           the last non-zero element on df.iloc[0,df_size[1]-1] =", df.iloc[0,col_last], '\n')
fv_data = np.array(df.iloc[:,0:fv_last+1])

Store the parameter array for later use

In [ ]:
X_par =  np.array(df.iloc[:,n_Cp_var:fv_last+1])
print('\n', "           number of sets of fv_data is ", len(fv_data), '\n')
print("           the last fv value in row #0 col #last is  fv_data[0,last] =", fv_data[0,fv_last], '\n')

----------------------------------------------------------------------------

reate X data

reating Y data

In [ ]:
X = fv_data
n_fv_data = X.shape[1]
Y = []
Y = df.iloc[:,fv_last+1:]
print("  \n \n    the last non-zero element on Y.iloc[0,row_last] =", Y.iloc[0,n_out_var-1], '\n')
Y = np.array(Y)
Y = Y.astype(float)
print("Last values in Y of row #0 \n \n y-coordinates \n \n", Y[0, 0:])

Create XY set and shuffle

In [ ]:
rows = X.shape[0]
cols = X.shape[1]
print('\n', "           rows =", rows, "cols =", cols, '\n')
assert fv_data.shape[0]==Y.shape[0], "# of fv sets equals # of test Labels"
XY_set = np.zeros((rows,cols))
print(" \n  XY set initialized       \n", XY_set.shape)
XY_set = X
print("    \n XY set only feature variables \n      ", XY_set.shape)
XY_set = np.hstack((XY_set, Y))
print("   \n XY set Feature and Output variable \n  \n     ", XY_set.shape)

Shuffle the training set<br>
p.random.shuffle(XY_set)<br>
rint("\n     XY_set after random shuffle  \n ", XY_set.shape, '\n')

ake the n_last  examples as the test set

In [ ]:
X_test = np.zeros((n_test,cols))
Y_test = np.zeros((n_test,cols))
Cp_test = np.zeros((n_test,cols))
n_last = n_test
X_test = XY_set[XY_set.shape[0]-1-n_last:XY_set.shape[0]-1,:cols]
Y_test = XY_set[XY_set.shape[0]-1-n_last:XY_set.shape[0]-1,cols:]
assert X_test.shape[0]==Y_test.shape[0], "# of test exampless equals # of test Labels"
Cp_test = XY_set[XY_set.shape[0]-1-n_last:XY_set.shape[0]-1,0:n_Cp_var]
Cp_test_set = np.concatenate((x_coord, Cp_test), axis=0)
X_test_par = XY_set[XY_set.shape[0]-1-n_last:XY_set.shape[0]-1,n_Cp_var:fv_last+1]
X_test_af = af_trng_set[XY_set.shape[0]-1-n_last:XY_set.shape[0]-1]

tring = X_test_af.iloc[0]<br>
----------------------------------------------------------------------------

Set up the CNN model

In [ ]:
X = XY_set[0:XY_set.shape[0]-n_last,:cols]
Y = XY_set[0:XY_set.shape[0]-n_last,cols:]
input_size = X.shape[1]
output_size = Y.shape[1]
print("        Shape of input vector X =", X.shape, '\n')
print("        Shape of output vector Y =", Y.shape, '\n')
assert X.shape[0]==Y.shape[0], "# of TEs equals # of Labels"

For reproducibility of results

In [ ]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
from keras import backend
 
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
idim = input_size
ilayer1 = idim+3
model = Sequential()
model.add(Dense(ilayer1, input_dim = idim, activation = 'sigmoid'))
model.add(Dense(int(ilayer1)+1, activation = 'sigmoid'))
model.add(Dense(int(ilayer1)+1, activation = 'sigmoid'))
model.add(Dense(int(ilayer1)+1, activation = 'sigmoid'))
model.add(Dense(output_size))
adam = Adam(lr=0.0005)
model.compile(adam, loss = "mse", metrics = ['accuracy', rmse])
print(" Printing model summary", model.summary(), '\n')

Save checkpoints during training<br>
Create a tf.keras.callbacks.ModelCheckpoint callback that saves weights only during training:

Create a callback that saves the model's weights

In [ ]:
checkpoint_path = "NACA_training/fv.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
fv_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                            save_weights_only=True, monitor='val_loss',
                            verbose=1, save_best_only=False)
epochs_init = 0
epochs_end = 10000

Model #1

In [ ]:
h = model.fit(x=X, y=Y, validation_split = 0.2, verbose =1, 
              batch_size = None, epochs = epochs_end, initial_epoch = epochs_init,
              steps_per_epoch = 10, validation_steps = 5, 
              validation_freq=10, callbacks=[fv_callback])

Note that validation_freq value must be less than epochs_end: otherwise get key error


<br>
#Model #2  # Errors in this case are very high<br>
h = model.fit(x=X, y=Y, validation_split = 0.1, verbose =1, <br>
              batch_size = None, initial_epoch =0, epochs = 10000, callbacks=[fv_callback])<br>
<br>
54545ls {checkpoint_dir}

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(h.history['accuracy'])
plt.title('accuracy')
plt.xlabel('epochs')
plt.legend('accuracy')
plt.figure(figsize=(12,4))
plt.plot(h.history['rmse'])
plt.title('rmse')
plt.xlabel('rmse')
plt.legend('rmse')
plt.figure(figsize=(12,4))
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.plot(h.history['rmse'])
plt.title('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss', 'rmse'])

In [ ]:
predictions = model.predict(X)
#print("predicted Y =(alfa, Re, thk) =", predictions)
#print("Actual Y =(alfa, Re, thk) =", Y)
#print("Actual Y =(alfa, Re, thk) =", Y)

In [ ]:
from sklearn.metrics import mean_squared_error
#print("root mean_squared_error % 5.2f" %(mean_squared_error(Y, predictions, squared=False)/len(Y)))
rmse = mean_squared_error(Y, predictions, squared=False)/len(Y)
print("root mean_squared_error % 8.4f" %(rmse))

----------------------------------------------------------------------------

In [ ]:
XX = np.zeros((n_train, input_size))
YY = np.zeros((n_train, input_size))
Cp_trng = np.zeros((n_train, input_size))
XX = X[n_beg:n_end,:]
YY = Y[n_beg:n_end,:]
Cp_trng = X[n_beg:n_end,0:n_Cp_var]
Cp_trng_set = np.concatenate((x_coord, Cp_trng), axis=0)
X_trng_par = X_par[n_beg:n_end,:]
X_trng_af = af_trng_set[n_beg:n_end]
X_trng_par_list = string_list_from_X_par(X_trng_par)

In [ ]:
print(" \n XX.shape \n" , XX.shape)
print(" \n YY.shape \n" , YY.shape)
print(" \n X_trng_par.shape \n" , X_trng_par.shape)
predictions = model.predict(XX)

In [ ]:
File_name = AF_family_name + "_results_trng_subset_yp_" + str(epochs_end) + ".xlsx"
print('\n', "Results on extracted training set n1:n2", '\n')
print_results(predictions, YY, X_trng_par_list, X_trng_af, Cp_trng_set, File_name)

Results on unseen test set

In [ ]:
XX = np.zeros((X_test.shape[0],X_test.shape[1] ))
YY = np.zeros((X_test.shape[0],X_test.shape[1] ))
XX = X_test
YY = Y_test
print(" \n XX.shape \n" , XX.shape)
print(" \n YY.shape \n" , YY.shape)
X_test_par_list = string_list_from_X_par(X_test_par)

In [ ]:
predictions = model.predict(XX)
File_name = AF_family_name + "_results_test_subset_yp_" + str(epochs_end) + ".xlsx"
print('\n', "Results on unseen test set ", '\n')
print_results(predictions, YY, X_test_par_list, X_test_af, Cp_test_set, File_name)

In [ ]:
plot_results(predictions, YY, X_test_par_list, X_test_af, Cp_test_set)

In [ ]:
print("\n \n exit")